In [1]:
# 0. Install dependencies if needed: !pip install -q plotly scikit-learn openai
# 1.  Book corpus ─────────────────────────────────────────────────
works = {
    "Post Office":
        "Autobiographical novel about Henry Chinaski's soul-crushing years as a postal clerk; "
        "gritty humor about work and survival.",
    "Ham on Rye":
        "Semi-autobiographical coming-of-age tale in Great-Depression Los Angeles.",
    "Women":
        "Older Chinaski's chaotic carousel of lovers amid late-life literary fame.",
    "Factotum":
        "Picaresque drift through menial jobs and bars in 1940s America.",
    "Pulp":
        "A meta-noir parody written as Bukowski faced his own mortality.",
    "Love Is a Dog from Hell":
        "Poems (1974-1977) on love, lust and despair, raw and direct.",
    "Tales of Ordinary Madness":
        "Short stories of Los-Angeles low-lifes, sex, booze and madness.",
    "Notes of a Dirty Old Man":
        "Underground newspaper columns: crude humour, candid confession, street life.",
    "You Get So Alone at Times…":
        "Mid-career poems on solitude, aging and small daily epiphanies.",
    "The Last Night of the Earth Poems":
        "Late-life poems confronting death, memory and meaning.",
}

titles, texts = list(works.keys()), list(works.values())


In [2]:
# ✪ 2.  Choose vectoriser ───────────────────────────────────────────
USE_OPENAI = False          # flip to True for full-semantic mode
if USE_OPENAI:
    import os, openai, numpy as np
    openai.api_key = os.getenv("OPENAI_API_KEY")
    embeds = [openai.embeddings.create(
                model="text-embedding-3-small", input=txt, encoding_format="float"
              ).data[0].embedding for txt in texts]
    X = np.array(embeds)
else:
    from sklearn.feature_extraction.text import TfidfVectorizer
    X = TfidfVectorizer(stop_words="english").fit_transform(texts).toarray()


In [3]:
# ✪ 3.  Reduce to 3-D ───────────────────────────────────────────────
from sklearn.decomposition import PCA
coords = PCA(n_components=3, random_state=42).fit_transform(X)


In [4]:
# ✪ 4.  Build Plotly figure ────────────────────────────────────────
import plotly.express as px, pandas as pd, pathlib, webbrowser
df = pd.DataFrame(coords, columns=["PC1","PC2","PC3"]); df["Title"] = titles

fig = px.scatter_3d(df, x="PC1", y="PC2", z="PC3",
                    text="Title", hover_name="Title",
                    title="Charles Bukowski – 3-D projection of key works")
fig.update_traces(marker=dict(size=6))
outfile = pathlib.Path("bukowski_3d.html")
fig.write_html(outfile, include_plotlyjs="cdn")
print(f"✓ Written → {outfile.resolve()}")
webbrowser.open(outfile.resolve().as_uri())


✓ Written → /Users/hunterbillhardt/idk/bukowski_3d_with_docs/bukowski_3d.html


True